In [13]:
import pandas as pd
import numpy as np
import time

model = {}
f = open('glove.txt','r')
for line in f:
    splitLine = line.split()
    word = splitLine[0]
    embedding = [float(val) for val in splitLine[1:]]
    model[word] = embedding
print "Done.",len(model)," words loaded!"




df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")





for df in [df_train,df_test]: 
    unix_cols = ['deadline','state_changed_at','launched_at','created_at']
    for x in unix_cols:
        col = []
        for i in df[x].values:
            unix_timestamp  = int(i)
            utc_time = time.gmtime(unix_timestamp)
            local_time = time.localtime(unix_timestamp)
            col.append(time.strftime("%Y-%m-%d %H:%M:%S", local_time))
        df[x] = col
    
    disable_communication = []    
    for i in df['disable_communication'].values:
        if str(i) == 'False':
            disable_communication.append(0)
        else:
            disable_communication.append(1)
    df['disable_communication'] = disable_communication
    
    col = []
    import datetime
    for i in range(len(df)):
        x = datetime.datetime.strptime(df['state_changed_at'].values[i],"%Y-%m-%d %H:%M:%S")- datetime.datetime.strptime(df['deadline'].values[i],"%Y-%m-%d %H:%M:%S")
        col.append(x.total_seconds())
    df['deadline-state_changed_at'] = col ############################
    
    col = []
    import datetime
    for i in range(len(df)):
        x = datetime.datetime.strptime(df['launched_at'].values[i],"%Y-%m-%d %H:%M:%S")- datetime.datetime.strptime(df['created_at'].values[i],"%Y-%m-%d %H:%M:%S")
        col.append(x.total_seconds())
    df['launched_at-created_at'] = col
    
    col = []
    import datetime
    for i in range(len(df)):
        x = datetime.datetime.strptime(df['deadline'].values[i],"%Y-%m-%d %H:%M:%S")- datetime.datetime.strptime(df['launched_at'].values[i],"%Y-%m-%d %H:%M:%S")
        col.append(x.total_seconds()/86400)
    df['deadline-launched_at'] = col
    
    
    col = []
    import datetime
    for i in range(len(df)):
        x = datetime.datetime.strptime(df['deadline'].values[i],"%Y-%m-%d %H:%M:%S")- datetime.datetime.strptime(df['created_at'].values[i],"%Y-%m-%d %H:%M:%S")
        col.append(x.total_seconds()/86400)
    df['deadline-created_at'] = col
    
    
    
for df in [df_train,df_test]:     
    keys_no = []
    keys_len = []
    keys = []
    for i in df['keywords'].values:
        k = i.split('-')
        keys.append(k)
        keys_len.append(len(k))
        keys_no.append(len(k))
    df['keys_no']  = keys_no
    df['keys']  = keys
    df['keys_len']  = keys_len

    words = []
    for i in range(len(df)):
        for j in df['keys'].values[i]:
                words.append(j)
    l = set(words) - set(model.keys())
    GLOVE = []
    #df_train_3['GLOVE'] = 0
    for i in range(len(df)):
        glocevecs = np.zeros(50)
        for j in df['keys'].values[i]:
            if j not in l:
                glocevecs = glocevecs+np.array(model[j])
        GLOVE.append(glocevecs)
    df['GLOVE'] = GLOVE

    for j in range(50):
        one = []
        for i in  df['GLOVE'].values:
            one.append(i[j])
        df[str(j)] = one   
        
        
        
df_train.to_csv('train1.csv')
df_test.to_csv('test1.csv')

Done. 400000  words loaded!
